In [1]:
import tensorrt as trt

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

def build_engine(onnx_path, engine_path):
    with trt.Builder(TRT_LOGGER) as builder, \
         builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)) as network, \
         trt.OnnxParser(network, TRT_LOGGER) as parser:
        
        # 解析 ONNX 模型
        with open(onnx_path, 'rb') as f:
            parser.parse(f.read())
        
        # 配置构建参数
        config = builder.create_builder_config()
        config.max_workspace_size = 1 << 30  # 1GB 工作空间
        config.set_flag(trt.BuilderFlag.FP16)  # 启用 FP16 加速（需 GPU 支持）
        
        # 构建并保存引擎
        engine = builder.build_engine(network, config)
        with open(engine_path, 'wb') as f:
            f.write(engine.serialize())
    return engine_path

# 转换模型（替换为你的 ONNX 路径）
build_engine("pet_detection_model.onnx", "pet_detection.engine")

AttributeError: 'tensorrt_bindings.tensorrt.IBuilderConfig' object has no attribute 'max_workspace_size'

In [4]:
import tensorrt as trt

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

def build_engine(onnx_path, engine_path):
    with trt.Builder(TRT_LOGGER) as builder, \
         builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)) as network, \
         trt.OnnxParser(network, TRT_LOGGER) as parser:
        
        # 解析 ONNX 模型
        with open(onnx_path, 'rb') as f:
            if not parser.parse(f.read()):
                print("ONNX 解析失败，错误信息：")
                for error in range(parser.num_errors):
                    print(parser.get_error(error))
                return None
        
        # 配置构建参数
        config = builder.create_builder_config()
        config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)  # 1GB
        
        # 启用 FP16 加速（需 GPU 支持）
        if builder.platform_has_fast_fp16:
            config.set_flag(trt.BuilderFlag.FP16)
        
        # 构建并保存引擎（适配新版本 API）
        serialized_engine = builder.build_serialized_network(network, config)
        if serialized_engine is None:
            print("引擎构建失败！")
            return None
            
        with open(engine_path, 'wb') as f:
            f.write(serialized_engine)
        return engine_path

# 转换模型
build_engine("pet_detection_model.onnx", "pet_detection.engine")

引擎构建失败！


In [5]:
import tensorrt as trt
import os

TRT_LOGGER = trt.Logger(trt.Logger.ERROR)  # 使用 ERROR 级别减少冗余日志

def build_engine(onnx_path, engine_path, fp16=True, workspace_size=2):
    """构建 TensorRT 引擎
    
    Args:
        onnx_path: ONNX 模型路径
        engine_path: 保存引擎的路径
        fp16: 是否启用 FP16 精度
        workspace_size: 工作空间大小（GB）
    """
    # 如果引擎已存在，直接返回
    if os.path.exists(engine_path):
        print(f"✅ 引擎已存在: {engine_path}")
        return engine_path
        
    # 创建构建器和网络
    builder = trt.Builder(TRT_LOGGER)
    network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
    parser = trt.OnnxParser(network, TRT_LOGGER)
    
    # 解析 ONNX 模型
    print(f"🔄 正在解析 ONNX 模型: {onnx_path}")
    with open(onnx_path, 'rb') as f:
        if not parser.parse(f.read()):
            print("❌ ONNX 解析失败，错误信息:")
            for error in range(parser.num_errors):
                print(f"  - {parser.get_error(error)}")
            return None
            
    print(f"✅ ONNX 解析成功，网络层数: {network.num_layers}")
    
    # 配置构建参数
    config = builder.create_builder_config()
    config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, workspace_size << 30)
    
    # 启用 FP16（如果支持）
    if fp16 and builder.platform_has_fast_fp16:
        print("🔄 启用 FP16 精度")
        config.set_flag(trt.BuilderFlag.FP16)
    else:
        print("🔄 使用 FP32 精度")
    
    # 构建并保存引擎
    print("🔄 正在构建 TensorRT 引擎...")
    serialized_engine = builder.build_serialized_network(network, config)
    
    if serialized_engine is None:
        print("❌ 引擎构建失败！")
        
        # 输出详细错误信息
        if network.num_layers == 0:
            print("  - 网络层数为0，ONNX解析可能失败")
        else:
            print(f"  - 成功解析网络，层数: {network.num_layers}")
            
        # 检查最后一层
        last_layer = network.get_layer(network.num_layers - 1)
        print(f"  - 最后一层: {last_layer.name}, 类型: {last_layer.type}")
        
        # 检查未连接的输出
        unconnected_outputs = False
        for i in range(network.num_layers):
            layer = network.get_layer(i)
            for j in range(layer.num_outputs):
                if not layer.get_output(j):
                    print(f"  - ❌ 第 {i} 层的第 {j} 个输出未连接")
                    unconnected_outputs = True
                    
        if not unconnected_outputs:
            print("  - 所有输出均已连接")
            
        return None
        
    # 保存引擎
    with open(engine_path, 'wb') as f:
        f.write(serialized_engine)
        
    print(f"✅ 引擎已成功保存到: {engine_path}")
    return engine_path

# 转换模型
build_engine(
    onnx_path="pet_detection_model.onnx",
    engine_path="pet_detection.engine",
    fp16=True,
    workspace_size=2
)

🔄 正在解析 ONNX 模型: pet_detection_model.onnx
✅ ONNX 解析成功，网络层数: 274
🔄 启用 FP16 精度
🔄 正在构建 TensorRT 引擎...
❌ 引擎构建失败！
  - 成功解析网络，层数: 274
  - 最后一层: /Squeeze_3, 类型: LayerType.SQUEEZE
  - 所有输出均已连接


In [7]:
import tensorrt as trt
import os
import onnx
import onnx_graphsurgeon as gs

TRT_LOGGER = trt.Logger(trt.Logger.ERROR)

def fix_onnx_squeeze(onnx_path, output_path):
    """修复 ONNX 模型中的 Squeeze 节点"""
    print(f"🔄 正在修复 ONNX 模型中的 Squeeze 节点: {onnx_path}")
    
    # 加载 ONNX 模型
    graph = gs.import_onnx(onnx.load(onnx_path))
    
    # 查找并替换 Squeeze 节点
    squeeze_count = 0
    for node in graph.nodes:
        if node.op == "Squeeze":
            # 记录 Squeeze 节点
            squeeze_count += 1
            print(f"  - 找到 Squeeze 节点: {node.name}")
            
            # 创建新的 Reshape 节点替代 Squeeze
            input_tensor = node.inputs[0]
            output_tensor = node.outputs[0]
            
            # 计算新形状（移除维度为1的轴）
            new_shape = []
            for i, dim in enumerate(input_tensor.shape):
                if dim != 1 or (i in node.attrs.get("axes", []) if "axes" in node.attrs else True):
                    new_shape.append(dim)
            
            # 创建形状张量
            shape_tensor = gs.Constant(name=f"{node.name}_shape", values=np.array(new_shape, dtype=np.int64))
            
            # 创建 Reshape 节点
            reshape_node = gs.Node(
                op="Reshape",
                name=f"{node.name}_reshape",
                inputs=[input_tensor, shape_tensor],
                outputs=[output_tensor]
            )
            
            # 添加新节点并移除原 Squeeze 节点
            graph.nodes.append(reshape_node)
            node.outputs = []
    
    if squeeze_count > 0:
        print(f"✅ 成功替换 {squeeze_count} 个 Squeeze 节点")
        # 清理并保存修改后的模型
        graph.cleanup().toposort()
        onnx.save(gs.export_onnx(graph), output_path)
        return output_path
    else:
        print("✅ 未发现需要修复的 Squeeze 节点")
        return onnx_path

def build_engine(onnx_path, engine_path, fp16=True, workspace_size=2):
    """构建 TensorRT 引擎"""
    # 如果引擎已存在，直接返回
    if os.path.exists(engine_path):
        print(f"✅ 引擎已存在: {engine_path}")
        return engine_path
        
    # 创建构建器和网络
    builder = trt.Builder(TRT_LOGGER)
    network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
    parser = trt.OnnxParser(network, TRT_LOGGER)
    
    # 解析 ONNX 模型
    print(f"🔄 正在解析 ONNX 模型: {onnx_path}")
    with open(onnx_path, 'rb') as f:
        if not parser.parse(f.read()):
            print("❌ ONNX 解析失败，错误信息:")
            for error in range(parser.num_errors):
                print(f"  - {parser.get_error(error)}")
            return None
            
    print(f"✅ ONNX 解析成功，网络层数: {network.num_layers}")
    
    # 配置构建参数
    config = builder.create_builder_config()
    config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, workspace_size << 30)
    
    # 启用 FP16（如果支持）
    if fp16 and builder.platform_has_fast_fp16:
        print("🔄 启用 FP16 精度")
        config.set_flag(trt.BuilderFlag.FP16)
        
        # 对所有 Squeeze 层强制使用 FP32
        for i in range(network.num_layers):
            layer = network.get_layer(i)
            if layer.type == trt.LayerType.SQUEEZE:
                print(f"  - 对 Squeeze 层 {layer.name} 使用 FP32 精度")
                layer.precision = trt.DataType.FLOAT
                layer.set_output_type(0, trt.DataType.FLOAT)
    
    # 构建并保存引擎
    print("🔄 正在构建 TensorRT 引擎...")
    serialized_engine = builder.build_serialized_network(network, config)
    
    if serialized_engine is None:
        print("❌ 引擎构建失败！")
        
        # 输出详细错误信息
        if network.num_layers == 0:
            print("  - 网络层数为0，ONNX解析可能失败")
        else:
            print(f"  - 成功解析网络，层数: {network.num_layers}")
            
        # 检查最后一层
        last_layer = network.get_layer(network.num_layers - 1)
        print(f"  - 最后一层: {last_layer.name}, 类型: {last_layer.type}")
        
        # 检查未连接的输出
        unconnected_outputs = False
        for i in range(network.num_layers):
            layer = network.get_layer(i)
            for j in range(layer.num_outputs):
                if not layer.get_output(j):
                    print(f"  - ❌ 第 {i} 层的第 {j} 个输出未连接")
                    unconnected_outputs = True
                    
        if not unconnected_outputs:
            print("  - 所有输出均已连接")
            
        return None
        
    # 保存引擎
    with open(engine_path, 'wb') as f:
        f.write(serialized_engine)
        
    print(f"✅ 引擎已成功保存到: {engine_path}")
    return engine_path

# 主流程
fixed_onnx_path = "pet_detection_model_fixed.onnx"
original_onnx_path = "pet_detection_model.onnx"

# 修复 ONNX 模型
fixed_onnx_path = fix_onnx_squeeze(original_onnx_path, fixed_onnx_path)

# 构建引擎
build_engine(
    onnx_path=fixed_onnx_path,
    engine_path="pet_detection.engine",
    fp16=True,
    workspace_size=2
)

🔄 正在修复 ONNX 模型中的 Squeeze 节点: pet_detection_model.onnx
  - 找到 Squeeze 节点: /Squeeze


TypeError: 'NoneType' object is not iterable